In [1]:
import numpy as np
import torch
import helper_functions
from FPLinQ import FP_optimize, FP
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

## Create Dataset

In [2]:
train_K = 20
test_K = 20
train_layouts = 20000
test_layouts = 2000
var_noise = 1
c = 1/np.sqrt(2)

In [3]:
train_channel_losses = np.abs(c * np.random.randn(train_layouts, train_K, train_K) + c * 1j * np.random.randn(train_layouts, train_K, train_K))
test_channel_losses = np.abs(c * np.random.randn(test_layouts, test_K, test_K) + c * 1j * np.random.randn(test_layouts, test_K, test_K))

#### Compute the label for training and test dataset via FPLinQ
The code for FPlinQ is copied from [2] https://github.com/willtop/Spatial_Deep_Learning_for_Wireless_Scheduling

In [4]:
## Get direct channel CSI
directLink_channel_losses = helper_functions.get_directLink_channel_losses(train_channel_losses)
## Get interference channel CSI
crossLink_channel_losses = helper_functions.get_crossLink_channel_losses(train_channel_losses)
Y = FP(np.ones([train_layouts, train_K]), train_channel_losses, var_noise, np.ones([train_layouts, train_K, 1]))
rates = helper_functions.compute_rates(var_noise, 
            Y, directLink_channel_losses, crossLink_channel_losses)
sr = np.mean(np.sum(rates,axis=1))
y_train = Y

In [5]:
directLink_channel_losses = helper_functions.get_directLink_channel_losses(test_channel_losses)
crossLink_channel_losses = helper_functions.get_crossLink_channel_losses(test_channel_losses)
Y = FP(np.ones([test_layouts, test_K]), test_channel_losses, var_noise, np.ones([test_layouts, test_K, 1]))
rates = helper_functions.compute_rates(var_noise, 
            Y, directLink_channel_losses, crossLink_channel_losses)
sr = np.mean(np.sum(rates,axis=1))
y_test = Y
print('Sum rate by FPlinQ:', sr)

Sum rate by FPlinQ: 2.3419111604536966


#### Create Pytorch Dataset
Please refer to https://docs.dgl.ai/guide/data.html for a tutorial for the usage of DGL dataset

In [6]:
class PCDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        'Initialization'
        n = data.shape[0]
        self.data = torch.tensor(np.sqrt(data.reshape(n,-1)), dtype = torch.float)
        self.labels = torch.tensor(labels, dtype = torch.float)

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        X = self.data[index]
        y = self.labels[index]
        return X, y

In [7]:
train_data = PCDataset(train_channel_losses, y_train)
test_data = PCDataset(test_channel_losses, y_test)

In [8]:
batch_size = 64
train_loader = DataLoader(train_data, batch_size, shuffle=True)
test_loader = DataLoader(test_data, 2000, shuffle=False)

## Build Neural Networks
The activation functions follows [1].

In [10]:
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, Sigmoid, BatchNorm1d as BN, ReLU6 as ReLU6
def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU(), BN(channels[i]))
        for i in range(1, len(channels))
    ])

In [11]:
class PCNet(torch.nn.Module):
    def __init__(self):
        super(PCNet, self).__init__()
        self.mlp = MLP([train_K**2, 400, 400])
        self.mlp = Seq(*[self.mlp,Seq(Lin(400, train_K), ReLU6())])

    def forward(self, data):
        return self.mlp(data)/6

In [12]:
def train(epoch):
    """ Train for one epoch. """
    model.train()
    loss_all = 0
    for batch_idx, (x_train, y_train) in enumerate(train_loader):
        #data = data.to(device)
        optimizer.zero_grad()
        output = model(x_train)
        loss = F.mse_loss(output, y_train)
        loss.backward()
        loss_all += loss.item() * len(x_train)
        optimizer.step()
    return loss_all / len(train_loader.dataset)

In [13]:
def test(loader, test_mode = False):
    model.eval()
    mse = nmse = 0
    sr = 0
    for (x_test, y_test) in loader:
        #data = data.to(device)
        output = model(x_test)
        loss = F.mse_loss(output, y_test)*train_K
        mse += loss.item() * len(x_test)
        if test_mode:
            nmse += (((output - y_test)**2).sum(axis = -1)/(y_test**2).sum(axis = -1)).sum().item()
            output[output > 0.5] = 1
            output[output < 0.5] = 0
            rates = helper_functions.compute_rates(var_noise, 
                    output.detach().numpy(), directLink_channel_losses, crossLink_channel_losses)
            sr += np.mean(np.sum(rates,axis=1)) * len(x_test)
    if test_mode:
        return mse / len(loader.dataset), nmse / len(loader.dataset), sr/len(loader.dataset)
    return mse / len(loader.dataset)

In [14]:
model = PCNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)

In [15]:
record = []
for epoch in range(0, 20):
    if(epoch % 1 == 0):
        with torch.no_grad():
            loss = test(train_loader)
            mse, nmse, rate = test(test_loader, True)
            print('Epoch {:03d}, Train Loss: {:.4f}, Val MSE: {:.4f}, Val NMSE: {:.4f}, Val Rate: {:.4f}'.format(
                epoch, loss, mse, nmse, rate))
            record.append([loss,mse,nmse,rate])
    train(epoch)
    scheduler.step()


Epoch 000, Train Loss: 4.7432, Val MSE: 4.7368, Val NMSE: 0.9844, Val Rate: 0.0000
Epoch 001, Train Loss: 1.8171, Val MSE: 1.9915, Val NMSE: 0.4265, Val Rate: 2.0861
Epoch 002, Train Loss: 1.6824, Val MSE: 1.8998, Val NMSE: 0.4065, Val Rate: 2.1032
Epoch 003, Train Loss: 1.5864, Val MSE: 1.8863, Val NMSE: 0.4044, Val Rate: 2.1122
Epoch 004, Train Loss: 1.4735, Val MSE: 1.8399, Val NMSE: 0.3937, Val Rate: 2.1251
Epoch 005, Train Loss: 1.4128, Val MSE: 1.8761, Val NMSE: 0.4016, Val Rate: 2.1251
Epoch 006, Train Loss: 1.3307, Val MSE: 1.9013, Val NMSE: 0.4072, Val Rate: 2.1195
Epoch 007, Train Loss: 1.2251, Val MSE: 1.8897, Val NMSE: 0.4046, Val Rate: 2.1210
Epoch 008, Train Loss: 1.1508, Val MSE: 1.9205, Val NMSE: 0.4122, Val Rate: 2.1225
Epoch 009, Train Loss: 1.0864, Val MSE: 1.9558, Val NMSE: 0.4189, Val Rate: 2.1153
Epoch 010, Train Loss: 1.0395, Val MSE: 2.0032, Val NMSE: 0.4292, Val Rate: 2.1106
Epoch 011, Train Loss: 0.9559, Val MSE: 2.0127, Val NMSE: 0.4330, Val Rate: 2.1136
Epoc

## References
[1] H. Sun, X. Chen, Q. Shi, M. Hong, X. Fu, and N. D. Sidiropoulos, “Learning to optimize: Training deep neural networks for interference management,” IEEE Trans. Signal Process., vol. 66, pp. 5438 – 5453, Oct. 2018.
[2]  W. Cui, K. Shen, and W. Yu, “Spatial deep learning for wireless scheduling,” IEEE J. Sel. Areas Commun., vol. 37, Jun. 2019.